In [1]:
from typing import Iterator, List
import gensim
from gensim import corpora, models

In [2]:
# String Constants for file path names
TOKENFILE = '../intermediate/titleabstract_tokens.txt'
TERMDICTFILE = '../models/titleabstract.dict'
TERMCORPUSFILE = '../models/titleabstract_corpus.mm'
TERMTFIDFMODELFILE = '../models/titleabstract.tfidf_model'
TERMTFIDFFILE = '../models/titleabstract_tfidf.mm'
CLASSFILE = '../intermediate/classifications_ipc.txt'
CLASSDICTFILE = '../models/classifications.dict'
CLASSCORPUSFILE = '../models/classifications_corpus.mm'
CLASSTFIDFMODELFILE = '../models/classifications.tfidf_model'
CLASSTFIDFFILE = '../models/classifications_tfidf.mm'

In [3]:
def get_tokens(infile: str) -> Iterator[List[str]]:
    '''Yield tokens split on whitespace from each newline terminated string(doc) in file (with newline stripped)
    Parameters
        infile: full path string to file containing strings
    Returns
        each string with newline stripped
    '''
    with open(infile, 'r', encoding='utf-8') as inf:
        for line in inf:
            yield line.strip().split()

In [4]:
def tokens2sparsevecs(infile: str, dictname: gensim.corpora.dictionary.Dictionary):
    return (dictname.doc2bow(tokens) for tokens in get_tokens(infile))

In [5]:
def save_corpus(filename: str, vecs, dictname: gensim.corpora.dictionary.Dictionary) -> None:
    '''Save a corpus of vectors in Matrix Market format
    Parameters
        filename: full path to file to save
        vecs: corpus to save
        dictname: gensim dictionary with id, word pairs
    '''
    corpora.MmCorpus.serialize(filename, vecs, id2word=dictname)

In [6]:
terms_dict = corpora.Dictionary(get_tokens(TOKENFILE))

In [7]:
terms_dict.save(TERMDICTFILE)

In [8]:
save_corpus(TERMCORPUSFILE, 
            tokens2sparsevecs(TOKENFILE, terms_dict), 
            terms_dict)

In [9]:
tfidf_model = models.TfidfModel(corpus=tokens2sparsevecs(TOKENFILE, terms_dict), 
                                id2word=terms_dict, 
                                normalize=True)

In [10]:
tfidf_model.save(TERMTFIDFMODELFILE)

In [11]:
tfidf_sv = tfidf_model[tokens2sparsevecs(TOKENFILE, terms_dict)]

In [12]:
save_corpus(TERMTFIDFFILE, 
            tfidf_sv, 
            terms_dict)

**Convert Classification Codes into Vectors**

In [13]:
cl_dict = corpora.Dictionary(get_tokens(CLASSFILE))

In [14]:
cl_dict.save(CLASSDICTFILE)

In [15]:
save_corpus(CLASSCORPUSFILE, 
            tokens2sparsevecs(CLASSFILE, cl_dict),
            cl_dict)

In [16]:
tfidf_model_cl = models.TfidfModel(corpus=tokens2sparsevecs(CLASSFILE, cl_dict), 
                                   id2word=cl_dict, 
                                   normalize=True)

In [17]:
tfidf_model_cl.save(CLASSTFIDFMODELFILE)

In [18]:
tfidf_sv_cl = tfidf_model[tokens2sparsevecs(CLASSFILE, cl_dict)]

In [19]:
save_corpus(CLASSTFIDFFILE,
            tfidf_sv_cl,
            cl_dict)